<a target="_blank" href="https://colab.research.google.com/github/vnstock-hq/vnstock_insider_guide/blob/main/demo/4-vnnews_v2_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# I. Cài đặt thư viện

In [ ]:
# Sao chép mã nguồn từ GitHub về Google Colab
!git clone https://github.com/vnstock-hq/vnstock_insider_guide
# Cài đặt gói phụ thuộc
%cd "/content/vnstock_insider_guide/assets"
!pip install vnii-0.0.7.tar.gz
# Chuyển đến thư mục vừa clone
%cd /content/vnstock_insider_guide/oneclick_installer
# Trao quyền thực thi cho file
!chmod +x linux_installer.run
# Chạy trình cài đặt
!./linux_installer.run

**Lưu ý: Bạn cần khởi động lại phiên làm việc sau khi cài đặt trên Google Colab**

![restar session](http://vnstocks.com/images/khoi-dong-lai-phien-lam-viec-colab.png)

# Minh họa

## Chương trình mẫu

In [ ]:
# Chuyển thư mục làm việc về nơi bạn muốn lưu dữ liệu tin tức
%cd /content
!vnstock-news-crawl

## Cập nhật tin tức từ 1 trang cụ thể

### Cập nhật tin tóm tắt từ Datafeed

In [1]:
import os
import pandas as pd
from vnstock_news.core.batch import BatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger

# Initialize logger
logger = setup_logger("export_feed_info", debug=True)

# Directory to store the CSV files
OUTPUT_DIR = "output/feed_info"

def fetch_feed_info(site_name: str, top_n: int = None, within: str = None):
    """
    Fetch sitemap or RSS feed data and save the brief information to a CSV file.

    Parameters:
        site_name (str): Name of the site (from SITES_CONFIG).
        top_n (int): Limit the number of articles to fetch.
        within (str): Time frame to filter articles (e.g., '1h', '1d', '7d').
    """
    try:
        # Initialize BatchCrawler
        logger.info(f"Processing site: {site_name}")
        crawler = BatchCrawler(site_name=site_name)

        # Determine the source (sitemap or RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif site_data.get("sitemap_url"):
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"No sources found for {site_name}. Skipping...")
            return

        # Fetch feed data (without crawling article details)
        logger.info(f"Fetching feed info from sources: {sources}")
        feed_df = crawler.prepare_feeder(sources)

        # Filter the data if time range or top_n is provided
        if not feed_df.empty:
            filtered_df = crawler.filter_feeder(feed_df, top_n=top_n, within=within)
        else:
            logger.warning(f"No data fetched for {site_name}")
            return

        # Output directory setup
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_feed_info.csv")

        # Save to CSV
        filtered_df.to_csv(output_file, index=False, encoding="utf-8-sig")
        logger.info(f"Saved feed info with {len(filtered_df)} records to {output_file}")

    except Exception as e:
        logger.error(f"Error processing site '{site_name}': {e}")


def main():
    """
    Main function to fetch and save feed information for all supported sites.
    """
    logger.info("Starting the feed info export process...")

    fetch_feed_info(site_name='vnexpress', top_n=20, within="1d")  # Update as needed

    logger.info("Feed info export process completed!")


if __name__ == "__main__":
    main()

2025-04-12 09:53:05,686 - Cache - INFO - Initialized memory cache system
2025-04-12 09:53:05,686 - export_feed_info - INFO - Starting the feed info export process...
2025-04-12 09:53:05,687 - export_feed_info - INFO - Processing site: vnexpress
2025-04-12 09:53:05,687 - Crawler - INFO - Using predefined configuration for site: vnexpress
2025-04-12 09:53:05,687 - vnstock_news.core.sitemap - INFO - Attempting to load sitemap directly from https://vnexpress.net/sitemap.xml
2025-04-12 09:53:05,885 - vnstock_news.core.sitemap - INFO - Sitemap loaded successfully using direct parsing.
2025-04-12 09:53:05,885 - export_feed_info - INFO - Fetching feed info from sources: ['https://vnexpress.net/rss/tin-moi-nhat.rss']
2025-04-12 09:53:05,886 - BatchCrawler - INFO - Checking source: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-04-12 09:53:05,887 - BatchCrawler - INFO - Parsing RSS feed: https://vnexpress.net/rss/tin-moi-nhat.rss
2025-04-12 09:53:05,887 - vnstock_news.core.rss - INFO - Fetching

### Cập nhật tin chi tiết từ trang cụ thể

In [ ]:
import os
import pandas as pd
import logging
from vnstock_news.core.batch import BatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger

# Initialize logger - fix the level parameter
# The setup_logger function likely expects a logging level constant, not a string
logger = setup_logger("update_news", debug=False)  # Bật/Tắt log thông báo cơ bản

# Directory to store the CSV files
OUTPUT_DIR = "output/cafebiz"

def fetch_and_save_news(site_name: str, top_n: int = 10, within: str = None):
    """
    Fetch news articles from the specified site and save them to a CSV file.

    Parameters:
        site_name (str): Name of the site (from SITES_CONFIG).
        top_n (int): Number of articles to fetch.
        within (str): Time frame to filter articles (e.g., '1h', '2d').
    """
    try:
        # Initialize the BatchCrawler
        logger.info(f"Processing site: {site_name}")
        crawler = BatchCrawler(site_name=site_name)

        # Determine the source (sitemap or RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif site_data.get("sitemap_url"):
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"No sources found for {site_name}. Skipping...")
            return

        # Fetch articles
        logger.info(f"Fetching articles from sources: {sources}")
        articles_df = crawler.fetch_articles(sources, top_n=top_n, within=within)

        # Output directory setup
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_news.csv")

        # Save to CSV
        articles_df.to_csv(output_file, index=False, encoding="utf-8-sig")
        logger.info(f"Saved {len(articles_df)} articles to {output_file}")

    except Exception as e:
        logger.error(f"Error processing site '{site_name}': {e}")


def main():
    """
    Main function to fetch and save news for all supported sites.
    """
    logger.info("Starting the news update process...")

    # Thay thế tên trang báo cần truy xuất dữ liệu
    fetch_and_save_news(site_name='cafebiz', top_n=100, within="1h")

    logger.info("News update process completed!")


if __name__ == "__main__":
    main()

## Cập nhật tin tức từ nhiều trang cùng lúc

### Cập nhật tóm tắt từ Datafeed

In [ ]:
import os
import pandas as pd
from vnstock_news.core.batch import BatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger

# Initialize logger
logger = setup_logger("export_feed_info", level="INFO")

# Directory to store the CSV files
OUTPUT_DIR = "output/feed_info"

def fetch_feed_info(site_name: str, top_n: int = None, within: str = None):
    """
    Fetch sitemap or RSS feed data and save the brief information to a CSV file.

    Parameters:
        site_name (str): Name of the site (from SITES_CONFIG).
        top_n (int): Limit the number of articles to fetch.
        within (str): Time frame to filter articles (e.g., '1h', '1d', '7d').
    """
    try:
        # Initialize BatchCrawler
        logger.info(f"Processing site: {site_name}")
        crawler = BatchCrawler(site_name=site_name)

        # Determine the source (sitemap or RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif site_data.get("sitemap_url"):
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"No sources found for {site_name}. Skipping...")
            return

        # Fetch feed data (without crawling article details)
        logger.info(f"Fetching feed info from sources: {sources}")
        feed_df = crawler.prepare_feeder(sources)

        # Filter the data if time range or top_n is provided
        if not feed_df.empty:
            filtered_df = crawler.filter_feeder(feed_df, top_n=top_n, within=within)
        else:
            logger.warning(f"No data fetched for {site_name}")
            return

        # Output directory setup
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_feed_info.csv")

        # Save to CSV
        filtered_df.to_csv(output_file, index=False, encoding="utf-8-sig")
        logger.info(f"Saved feed info with {len(filtered_df)} records to {output_file}")

    except Exception as e:
        logger.error(f"Error processing site '{site_name}': {e}")


def main():
    """
    Main function to fetch and save feed information for all supported sites.
    """
    logger.info("Starting the feed info export process...")

    # Iterate through all supported sites
    for site_name in SITES_CONFIG.keys():
        fetch_feed_info(site_name=site_name, top_n=20, within="1d")  # Update as needed

    logger.info("Feed info export process completed!")


if __name__ == "__main__":
    main()

### Cập nhật tin chi tiết từ nhiều trang

In [ ]:
import os
import pandas as pd
import logging
from vnstock_news.core.batch import BatchCrawler
from vnstock_news.config.sites import SITES_CONFIG
from vnstock_news.utils.logger import setup_logger

# Initialize logger - fix the level parameter
# The setup_logger function likely expects a logging level constant, not a string
logger = setup_logger("update_news", debug=False)  # Using debug=False for INFO level

# If your setup_logger was modified to accept an integer level:
# logger = setup_logger("update_news", level=logging.INFO)

# Directory to store the CSV files
OUTPUT_DIR = "output/news"

def fetch_and_save_news(site_name: str, top_n: int = 10, within: str = None):
    """
    Fetch news articles from the specified site and save them to a CSV file.

    Parameters:
        site_name (str): Name of the site (from SITES_CONFIG).
        top_n (int): Number of articles to fetch.
        within (str): Time frame to filter articles (e.g., '1h', '2d').
    """
    try:
        # Initialize the BatchCrawler
        logger.info(f"Processing site: {site_name}")
        crawler = BatchCrawler(site_name=site_name)

        # Determine the source (sitemap or RSS)
        site_data = SITES_CONFIG.get(site_name)
        sources = []
        if site_data.get("rss"):
            sources = site_data["rss"]["urls"]
        elif site_data.get("sitemap_url"):
            sources = [site_data["sitemap_url"]]

        if not sources:
            logger.error(f"No sources found for {site_name}. Skipping...")
            return

        # Fetch articles
        logger.info(f"Fetching articles from sources: {sources}")
        articles_df = crawler.fetch_articles(sources, top_n=top_n, within=within)

        # Output directory setup
        os.makedirs(OUTPUT_DIR, exist_ok=True)
        output_file = os.path.join(OUTPUT_DIR, f"{site_name}_news.csv")

        # Save to CSV
        articles_df.to_csv(output_file, index=False, encoding="utf-8-sig")
        logger.info(f"Saved {len(articles_df)} articles to {output_file}")

    except Exception as e:
        logger.error(f"Error processing site '{site_name}': {e}")


def main():
    """
    Main function to fetch and save news for all supported sites.
    """
    logger.info("Starting the news update process...")

    # Iterate through all supported sites
    for site_name in SITES_CONFIG.keys():
        fetch_and_save_news(site_name=site_name, top_n=10, within="1d")  # Update as needed

    logger.info("News update process completed!")


if __name__ == "__main__":
    main()